# Carga de JSON a CSV

In [44]:
import json
import pandas as pd

In [45]:
# Apertura del archivo
f = open('partido.json')

# Conversión en diccionario
data = json.load(f)
# Cierre del archivo
f.close()

In [46]:
print(data)

{'statistics': [{'period': 'ALL', 'groups': [{'groupName': 'Expected', 'statisticsItems': [{'name': 'Expected goals', 'home': '0.72', 'away': '0.27', 'compareCode': 1}, {'name': 'Expected assists', 'home': '0.74', 'away': '0.43', 'compareCode': 1}]}, {'groupName': 'Possession', 'statisticsItems': [{'name': 'Ball possession', 'home': '58%', 'away': '42%', 'compareCode': 1}]}, {'groupName': 'Shots', 'statisticsItems': [{'name': 'Total shots', 'home': '9', 'away': '7', 'compareCode': 1}, {'name': 'Shots on target', 'home': '5', 'away': '0', 'compareCode': 1}, {'name': 'Shots off target', 'home': '3', 'away': '3', 'compareCode': 3}, {'name': 'Blocked shots', 'home': '1', 'away': '4', 'compareCode': 2}]}, {'groupName': 'TVData', 'statisticsItems': [{'name': 'Corner kicks', 'home': '2', 'away': '2', 'compareCode': 3}, {'name': 'Offsides', 'home': '4', 'away': '1', 'compareCode': 1}, {'name': 'Fouls', 'home': '13', 'away': '23', 'compareCode': 2}, {'name': 'Yellow cards', 'home': '1', 'away':

Normalize no nos satisface

In [47]:
df = pd.json_normalize(data, sep='_')
df.head()

,statistics
0,"[{'period': 'ALL', 'groups': [{'groupName': 'E..."


La información se encuentra en una única clave dentro del diccionario llamada statistics

In [48]:
data = data["statistics"]

Dentro de data, ahora hay una lista de tres elementos. 
Tomamos el primero, que corresponde al partido completo. Los otros dos son para el primer y segundo tiempo.

In [49]:
all = data[0]["groups"]

Ahora json-normalize hace mejor trabajo

In [50]:
json_all =json.dumps(all)
df = pd.json_normalize(json.loads(json_all), sep='_')
print(df)

     groupName                                    statisticsItems
0     Expected  [{'name': 'Expected goals', 'home': '0.72', 'a...
1   Possession  [{'name': 'Ball possession', 'home': '58%', 'a...
2        Shots  [{'name': 'Total shots', 'home': '9', 'away': ...
3       TVData  [{'name': 'Corner kicks', 'home': '2', 'away':...
4  Shots extra  [{'name': 'Big chances', 'home': '3', 'away': ...
5       Passes  [{'name': 'Passes', 'home': '530', 'away': '38...
6        Duels  [{'name': 'Dribbles', 'home': '8/17 (47%)', 'a...
7    Defending  [{'name': 'Tackles', 'home': '22', 'away': '18...


La columna statisticsItems contiene una lista de listas. Si alineo los elementos en una sola lista unidimensional...

In [51]:
statistics_items = df["statisticsItems"].tolist()
statistics_items

items = []
for elemento in statistics_items:
    items.extend(elemento)
print(items)

[{'name': 'Expected goals', 'home': '0.72', 'away': '0.27', 'compareCode': 1}, {'name': 'Expected assists', 'home': '0.74', 'away': '0.43', 'compareCode': 1}, {'name': 'Ball possession', 'home': '58%', 'away': '42%', 'compareCode': 1}, {'name': 'Total shots', 'home': '9', 'away': '7', 'compareCode': 1}, {'name': 'Shots on target', 'home': '5', 'away': '0', 'compareCode': 1}, {'name': 'Shots off target', 'home': '3', 'away': '3', 'compareCode': 3}, {'name': 'Blocked shots', 'home': '1', 'away': '4', 'compareCode': 2}, {'name': 'Corner kicks', 'home': '2', 'away': '2', 'compareCode': 3}, {'name': 'Offsides', 'home': '4', 'away': '1', 'compareCode': 1}, {'name': 'Fouls', 'home': '13', 'away': '23', 'compareCode': 2}, {'name': 'Yellow cards', 'home': '1', 'away': '2', 'compareCode': 2}, {'name': 'Big chances', 'home': '3', 'away': '0', 'compareCode': 1}, {'name': 'Big chances missed', 'home': '2', 'away': '0', 'compareCode': 1}, {'name': 'Hit woodwork', 'home': '1', 'away': '0', 'compareCo

Ya tengo una lista de diccionarios, todos iguales. Si lo convierto en un dataframe ya tengo los datos numéricos en celdas individuales

In [52]:
datos_final = pd.DataFrame(items)
datos_final = datos_final.T
print(datos_final)

                         0                 1                2            3   \
name         Expected goals  Expected assists  Ball possession  Total shots   
home                   0.72              0.74              58%            9   
away                   0.27              0.43              42%            7   
compareCode               1                 1                1            1   

                          4                 5              6             7   \
name         Shots on target  Shots off target  Blocked shots  Corner kicks   
home                       5                 3              1             2   
away                       0                 3              4             2   
compareCode                1                 3              2             3   

                   8      9   ...               19           20          21  \
name         Offsides  Fouls  ...  Accurate passes   Long balls     Crosses   
home                4     13  ...        452 (85%)

Convierto la primera fila en nombres de columnas

In [53]:
datos_final.columns = datos_final.iloc[0]
datos_final = datos_final[1:]
print(datos_final)

name        Expected goals Expected assists Ball possession Total shots  \
home                  0.72             0.74             58%           9   
away                  0.27             0.43             42%           7   
compareCode              1                1               1           1   

name        Shots on target Shots off target Blocked shots Corner kicks  \
home                      5                3             1            2   
away                      0                3             4            2   
compareCode               1                3             2            3   

name        Offsides Fouls  ... Accurate passes   Long balls     Crosses  \
home               4    13  ...       452 (85%)  44/79 (56%)  2/15 (13%)   
away               1    23  ...       299 (78%)  24/52 (46%)  7/12 (58%)   
compareCode        1     2  ...               1            1           2   

name           Dribbles Possession lost Duels won Aerials won Tackles  \
home         8/17 (4

Extraigo las dos filas (home y away)

In [54]:
datos_final_home = datos_final.loc["home"]
datos_final_away = datos_final.loc["away"]

Las convierto en DataFrames (las filas, al ser unidimensionales se convierten automáticamente en "Series" y necesito que sean DataFrames para unirlas)

In [55]:
datos_final_home = pd.DataFrame(datos_final_home).transpose()
datos_final_away = pd.DataFrame(datos_final_away).transpose()

Elimino los índices para que al usar join me ponga los datos en la misma fila (fila = 0)

In [ ]:
datos_final_home = datos_final_home.reset_index(drop=True)
datos_final_away = datos_final_away.reset_index(drop=True)

...y hago join

In [56]:
final = datos_final_home.join(datos_final_away, lsuffix='_home', how='outer', rsuffix='_away')

In [57]:
# Guardar final en un csv
final.to_csv("final.csv")